In [3]:
from datamodule import CT_Datamodule
from torchvision.transforms import functional as TF
import torch

In [4]:
d = CT_Datamodule("Dataset", num_workers=0)

In [5]:
d.prepare_data()
d.setup("fit")

In [7]:
def calculate_stats(loader: torch.utils.data.DataLoader):
    """
    Calculate the mean and standard deviation of the data in the loader

    Args:
        loader (torch.utils.data.DataLoader): The data loader

    Returns:
        Tuple[torch.Tensor]: The channel-wise mean and standard deviation of the data
    """
    psum = torch.tensor([0.0] * loader.dataset[0][2][0].shape[0])
    psum_sq = torch.tensor([0.0] * loader.dataset[0][2][0].shape[0])

    for _, _, inputs, _ in loader:
        inputs = inputs[0]
        psum += inputs.sum(axis=[0, 2, 3])
        psum_sq += (inputs**2).sum(axis=[0, 2, 3])

    count = (
        len(loader.dataset)
        * loader.dataset[0][2][0].shape[1]
        * loader.dataset[0][2][0].shape[2]
    )

    total_mean = psum / count
    total_var = (psum_sq / count) - (total_mean**2)
    total_std = torch.sqrt(total_var)
    return total_mean, total_std

In [8]:
calculate_stats(d.train_dataloader())

(tensor([0.1737, 0.1737, 0.1737]), tensor([0.2584, 0.2584, 0.2584]))

In [6]:
import torch

samples_reg = torch.zeros(4)
samples_tum = torch.zeros(4)

for i, j, _, _ in d.train:
    samples_reg += i
    samples_tum += j

for i, j, _, _ in d.val:
    samples_reg += i
    samples_tum += j

In [11]:
samples_tum

tensor([ 99., 362., 114.,  75.], dtype=torch.float64)

In [12]:
samples_reg


tensor([305., 201.,  80.,  64.], dtype=torch.float64)

In [20]:
m= torch.hub.load("facebookresearch/dinov2", "dinov2_vits14")

Using cache found in C:\Users\Owais Makroo/.cache\torch\hub\facebookresearch_dinov2_main


In [16]:
m.con

NestedTensorBlock(
  (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
  (attn): MemEffAttention(
    (qkv): Linear(in_features=384, out_features=1152, bias=True)
    (attn_drop): Dropout(p=0.0, inplace=False)
    (proj): Linear(in_features=384, out_features=384, bias=True)
    (proj_drop): Dropout(p=0.0, inplace=False)
  )
  (ls1): LayerScale()
  (drop_path1): Identity()
  (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
  (mlp): Mlp(
    (fc1): Linear(in_features=384, out_features=1536, bias=True)
    (act): GELU(approximate='none')
    (fc2): Linear(in_features=1536, out_features=384, bias=True)
    (drop): Dropout(p=0.0, inplace=False)
  )
  (ls2): LayerScale()
  (drop_path2): Identity()
)

In [17]:
m

DinoVisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 384, kernel_size=(14, 14), stride=(14, 14))
    (norm): Identity()
  )
  (blocks): ModuleList(
    (0-11): 12 x NestedTensorBlock(
      (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (attn): MemEffAttention(
        (qkv): Linear(in_features=384, out_features=1152, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): LayerScale()
      (drop_path1): Identity()
      (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=384, out_features=1536, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=1536, out_features=384, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
      (ls2): LayerScale()
      (drop_path2): Identity()
    )
  )
  (n

In [1]:
from model import Classifier

In [2]:
m = Classifier(1e-3, 2e-5, 1)

Using cache found in C:\Users\Owais Makroo/.cache\torch\hub\facebookresearch_dinov2_main
C:\Users\Owais Makroo/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
C:\Users\Owais Makroo/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
C:\Users\Owais Makroo/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


In [3]:
m

Classifier(
  (conLoss): SupervisedContrastiveLoss()
  (ceLoss_ppgl): CrossEntropyLoss()
  (ceLoss_region): CrossEntropyLoss()
  (multiclass_accuracy): MulticlassAccuracy()
  (total_accuracy): MulticlassAccuracy()
  (multiclass_f1): MulticlassF1Score()
  (total_f1): MulticlassF1Score()
  (multiclass_auc): MulticlassAUROC()
  (total_auc): MulticlassAUROC()
  (multiclass_rec): MulticlassRecall()
  (total_rec): MulticlassRecall()
  (vit): ViTFeatureExtractor(
    (base_model): DinoVisionTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 384, kernel_size=(14, 14), stride=(14, 14))
        (norm): Identity()
      )
      (blocks): ModuleList(
        (0-11): 12 x NestedTensorBlock(
          (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
          (attn): MemEffAttention(
            (qkv): Module(
              (original): Linear(in_features=384, out_features=1152, bias=True)
              (lora): LoRALayer(
                (lora_down): Linear(in_featu